**Install Packages and Setup Variables**

In [ ]:
!pip install -q llama-index==0.14.0 openai==1.107.0 llama-index-finetuning==0.4.0 llama-index-embeddings-huggingface==0.6.0 jedi==0.19.2\
                llama-index-embeddings-cohere==0.6.0 cohere==5.17.0 llama-index-readers-web==0.5.1 tiktoken==0.11.0 chromadb==1.0.20 \
                llama-index-vector-stores-chroma==0.5.2 llama-index-llms-google-genai==0.3.0 llama-index-llms-openai==0.5.4 \
                llama-index-embeddings-instructor==0.4.0

In [ ]:
import os

# Set the following API Keys in the Python environment. Will be used later.
# os.environ["OPENAI_API_KEY"] = "<YOUR_OPENAI_KEY>"
# os.environ["COHERE_API_KEY"] = "<YOUR_OPENAI_KEY>"
# os.environ["GOOGLE_API_KEY"] = "<YOUR_OPENAI_KEY>"

from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["COHERE_API_KEY"] = userdata.get('COHERE_API_KEY')
os.environ["GOOGLE_API_KEY"] = userdata.get('Google_api_key')

In [ ]:
# Allows running asyncio in environments with an existing event loop, like Jupyter notebooks.

import nest_asyncio

nest_asyncio.apply()

**Load a Model**

In [ ]:
from llama_index.embeddings.cohere import CohereEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

Settings.llm = OpenAI(model="gpt-5", additional_kwargs={"reasoning_effort": "minimal"})

Settings.embed_model = CohereEmbedding(model_name="embed-english-v3.0",api_key=os.environ["COHERE_API_KEY"])

In [ ]:
# # If you have Paid Gemini API Key

# from llama_index.llms.google_genai import GoogleGenAI
# llm = GoogleGenAI(model="gemini-2.5-flash", temperature=1)

**Create a VectoreStore**

In [ ]:
import chromadb

# create a client and a new collection
# chromadb.EphemeralClient saves data in-memory.

chroma_client = chromadb.PersistentClient(path="./mini-llama-articles")
chroma_collection = chroma_client.create_collection("mini-llama-articles")

In [ ]:
from llama_index.vector_stores.chroma import ChromaVectorStore

# Define a storage context object using the created vector database.
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)


## **Load the Dataset (CSV)**

The dataset includes several articles from the TowardsAI blog, which provide an in-depth explanation of the LLaMA2 model. Read the dataset as a long string.

In [ ]:
!curl -o ./mini-llama-articles.csv https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/mini-llama-articles.csv

**Read File**

In [ ]:
import csv

rows = []

# Load the file as a JSON
with open("./mini-llama-articles.csv", mode="r", encoding="utf-8") as file:
    csv_reader = csv.reader(file)

    for idx, row in enumerate(csv_reader):
        if idx == 0:
            continue
            # Skip header row
        rows.append(row)

# The number of characters in the dataset.
len(rows)

**Convert to Document obj**

In [ ]:
from llama_index.core import Document

# Convert the chunks to Document objects so the LlamaIndex framework can process them.
documents = [
    Document(text=row[1], metadata={"title": row[0], "url": row[2], "source_name": row[3]})
    for row in rows[:12] # To Avoid Rate limit error in Cohere Trial API Key
]

**Transforming**

In [ ]:
from llama_index.core.text_splitter import TokenTextSplitter

# Define the splitter object that split the text into segments with 512 tokens,
# with a 128 overlap between the segments.
text_splitter = TokenTextSplitter(separator=" ", chunk_size=512, chunk_overlap=128)

There are two options to use the Cohere embeddings:

- input_type="search_document": Employ this option for texts (documents) intended for storage in your vector database.

- input_type="search_query": Use this when issuing search queries to locate the most related documents within your vector database.

In [ ]:
from llama_index.core.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    KeywordExtractor,
)
from llama_index.embeddings.cohere import CohereEmbedding
from llama_index.core.ingestion import IngestionPipeline


# Create the pipeline to apply the transformation on each chunk,
# and store the transformed text in the chroma vector store.
pipeline = IngestionPipeline(
    transformations=[
        text_splitter,
        QuestionsAnsweredExtractor(questions=3),
        SummaryExtractor(summaries=["prev", "self"]),
        KeywordExtractor(keywords=10),
        CohereEmbedding(model_name="embed-english-v3.0", input_type="search_document", api_key=os.environ["COHERE_API_KEY"]),
    ],
    vector_store=vector_store,
)

# Run the transformation pipeline.
nodes = pipeline.run(documents=documents, show_progress=True)

In [ ]:
len(nodes)

In [ ]:
len(nodes[0].embedding)

In [ ]:
# Compress the vector store directory to a zip file to be able to download and use later.
!zip -r vectorstore_cohere.zip mini-llama-articles

**Load Indexes**

If you have already uploaded the zip file for the vector store checkpoint, please uncomment the code in the following cell block to extract its contents. After doing so, you will be able to load the dataset from local storage.

In [ ]:
# Load the vector store from the local storage.
db = chromadb.PersistentClient(path="./mini-llama-articles")
chroma_collection = db.get_or_create_collection("mini-llama-articles")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

In [ ]:
from llama_index.core import VectorStoreIndex

# Create the index based on the vector store.
index = VectorStoreIndex.from_vector_store(vector_store)

**Query Dataset**

In [ ]:
# Define a query engine that is responsible for retrieving related pieces of text,
# and using a LLM to formulate the final answer.
query_engine = index.as_query_engine(similarity_top_k=5)

res = query_engine.query("How many parameters LLaMA 2 model has?")

In [ ]:
res.response

In [ ]:
# Show the retrieved nodes
for src in res.source_nodes:
    print("Node ID\t", src.node_id)
    print("Title\t", src.metadata["title"])
    print("Text\t", src.text)
    print("Score\t", src.score)
    print("-_" * 20)

**Evaluate**

In [ ]:
from llama_index.core.evaluation import generate_question_context_pairs
from llama_index.llms.openai import OpenAI

# Create questions for each segment. These questions will be used to
# assess whether the retriever can accurately identify and return the
# corresponding segment when queried.

llm_gpt = OpenAI(model="gpt-5", additional_kwargs={"reasoning_effort": "minimal"})

rag_eval_dataset = generate_question_context_pairs(
    nodes, llm=llm_gpt, num_questions_per_chunk=1
)

# We can save the evaluation dataset as a json file for later use.
rag_eval_dataset.save_json("./rag_eval_dataset_cohere.json")

If you have uploaded the generated question JSON file, please uncomment the code in the next cell block. This will avoid the need to generate the questions manually, saving you time and effort.

In [ ]:
import pandas as pd

#  A simple function to show the evaluation result.
def display_results_retriever(name, eval_results):
    """Display results from evaluate."""

    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)

    hit_rate = full_df["hit_rate"].mean()
    mrr = full_df["mrr"].mean()

    metric_df = pd.DataFrame(
        {"Retriever Name": [name], "Hit Rate": [hit_rate], "MRR": [mrr]}
    )

    return metric_df

In [29]:
from llama_index.core.evaluation import RetrieverEvaluator
import time

# We can evaluate the retievers with different top_k values.
for i in [2, 4, 6, 8, 10]:
    retriever = index.as_retriever(similarity_top_k=i)
    retriever_evaluator = RetrieverEvaluator.from_metric_names(
        ["mrr", "hit_rate"], retriever=retriever
    )

    eval_results = await retriever_evaluator.aevaluate_dataset(rag_eval_dataset)
    print(display_results_retriever(f"Retriever top_{i}", eval_results))

    time.sleep(70) # To Avaoid Rate limit error in Trial Cohere API Key

In [30]:
from llama_index.core.evaluation import (
    RelevancyEvaluator,
    FaithfulnessEvaluator,
    BatchEvalRunner,
)
from llama_index.llms.openai import OpenAI
llm_gpt_mini = OpenAI(model="gpt-5-mini", additional_kwargs={"reasoning_effort": "minimal"})

for i in [2, 4, 6, 8, 10]:
    # Set Faithfulness and Relevancy evaluators
    query_engine = index.as_query_engine(similarity_top_k=i)

    faithfulness_evaluator = FaithfulnessEvaluator(llm=llm_gpt_mini)
    relevancy_evaluator = RelevancyEvaluator(llm=llm_gpt_mini)

    # Run evaluation
    queries = list(rag_eval_dataset.queries.values())
    batch_eval_queries = queries[:20]

    runner = BatchEvalRunner(
        {"faithfulness": faithfulness_evaluator, "relevancy": relevancy_evaluator},
        workers=8,
    )
    eval_results = await runner.aevaluate_queries(
        query_engine, queries=batch_eval_queries
    )
    faithfulness_score = sum(
        result.passing for result in eval_results["faithfulness"]
    ) / len(eval_results["faithfulness"])
    print(f"top_{i} faithfulness_score: {faithfulness_score}")

    relevancy_score = sum(
        result.passing for result in eval_results["faithfulness"]
    ) / len(eval_results["relevancy"])
    print(f"top_{i} relevancy_score: {relevancy_score}")
    print("-_" * 10)